In [ ]:
import pandas
import re

# Read data

In [ ]:
def toInt(s, d = 1):
    try:
        s = str(int(s))
        return addZeros(s, d)
    except:
        return "X"
def addZeros(s, d):
    s = str(s)
    return '0' * (d - len(str(s))) + s 

def standardiseVenezuelan(s):
    s = str(s)
    s = s.lower().replace('ven', 'v')
    s = s.replace('v', 'VEN')
    return s

def addID01(df):
    placeId = '806001061-8'
    placeId = addZeros(placeId, 20)
    df['ID01'] = placeId + df['VAR_0019'].map(lambda s: addZeros(s,20))  \
                                              + df['VAR_0040'].map(lambda s: toInt(s, 2)) + \
                                            df['VAR_0286'].map(toInt)
    

In [ ]:
nivelN1 = pandas.read_csv('Data/nivelN1_manual.csv')
nivelN1['VAR_0019'] = nivelN1['VAR_0019'].map(standardiseVenezuelan)
addID01(nivelN1)
nivelN1.index = nivelN1.ID01
nivelN1 = nivelN1.loc[~nivelN1.index.duplicated(keep='first')]

automatic = pandas.read_csv('Data/results.csv')
automatic['VAR_0019'] = automatic['VAR_0019'].map(standardiseVenezuelan)
addID01(automatic)
def transformIDGP(s):
    """
    Remove 1 to gestas previas
    """
    s = re.match('(.*)([0-9][0-9])([0-9])', s)
    return s.group(1) + '%02d' % (int(s.group(2)) -1) + s.group(3)
automatic = automatic[automatic.ID01.map(lambda s: 'X' not in s)]
automatic.ID01 = automatic.ID01.map(transformIDGP)
automatic.index = automatic.ID01

In [ ]:
len(set(nivelN1.columns))

In [ ]:
pathFolder = 'Casos2019'
casos = pandas.read_csv(pathFolder + '/casos.csv', index_col = 0)
pacientes = pandas.read_csv(pathFolder + '/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv(pathFolder + '/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

#diagnosis = pandas.read_csv('Venezolanas2/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv(pathFolder + '/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)
registrosByCaso = registros.groupby('Caso')

entriesInfirmery = pandas.read_csv(pathFolder + '/enfermeriaMedidas.csv', index_col = 0)
entriesInfirmeryByCase = entriesInfirmery.groupby('IdAdmision')

# Explore differences

- Explore differences in the values, and which values I could not recover
- Some of the registers are missing, but the matching can still be done (it is in database transformer). There are ~ 90 missing records that I could not find.

In [ ]:
import datetime
nivelN1['dayOperation'] = nivelN1.VAR_0183.map(lambda s: datetime.datetime.strptime(s, '%m/%d/%y %H:%M:%S') if s== s else s)
automatic['dayOperation'] = automatic.VAR_0183.map(lambda s: datetime.datetime.strptime(s.split()[0], '%Y-%m-%d') if s == s else s)
automaticByDay = automatic.groupby('dayOperation')

In [ ]:
c = 0
errors = []
automatic2manual = {}
for k, i in nivelN1.iterrows():
    if not isinstance(i.VAR_0183, str) or '/19 ' not in i.VAR_0183 :
        continue
        
    if i.ID01 not in automatic.index:
        c += 1
        errors.append((i.ID01, i.VAR_0019, i.dayOperation))
        j = i
    else:
        automatic2manual[i.ID01] = i.ID01
print (len(errors), len(automatic2manual))

In [ ]:
import pickle
with open('Data/notFoundInAutomatic.pkl' , 'wb') as file:
    pickle.dump(errors, file)

# Check differences

In [ ]:
dfNamesVars = pandas.read_excel('parsingData/variables_sip.xlsx')
renameDict = {k : v for  k, v in zip(dfNamesVars['Unnamed: 2'].values, dfNamesVars['Unnamed: 3'].values)}
renameDict['VAR_0189'] = 'Corticoides / semana de inicio'
groupedBy = dfNamesVars.groupby('Unnamed: 2')
def renameDatabaseSIP(df):
    df.columns = df.columns.map(lambda s: s if s not in renameDict.values() else s + 'Self')
    df.columns = df.columns.map(lambda s: s if s not in renameDict else renameDict[s])
    for g in groupedBy.groups:
        if len(groupedBy.get_group(g)) == 1 or renameDict[g] not in df:
            continue
        colname = renameDict[g]
        for (_, r) in groupedBy.get_group(g)[['Unnamed: 4', 'Unnamed: 5']].iterrows():
            option = r['Unnamed: 4']
            value = r['Unnamed: 5']
            df.loc[df[colname] == option, colname] = value
renameDatabaseSIP(nivelN1)
renameDatabaseSIP(automatic)

In [ ]:
renameDict['VAR_0283']

In [ ]:
commonCols = set(nivelN1.columns).intersection(automatic.columns)
print('Common columns:', len(commonCols), len(nivelN1.columns))

In [ ]:
nivelN1 = nivelN1.loc[~nivelN1.index.duplicated(keep='first')]

In [ ]:
for i, k in enumerate(automatic2manual):
    print()
    print('Differences :', k)
    for c in commonCols:
        if automatic.loc[k, c] != nivelN1.loc[k, c]:
            print('VAR %s : auto= %s  , manual= %s' % (c, automatic.loc[k, c], nivelN1.loc[k, c]))
    if i == 3:
        break

In [ ]:
def hclin2pid(hClin):
    return pacientes.loc[pacientes.Identificacion == hClin].index[0]

In [ ]:
automatic[automatic.VAR_0019 == errors[25]]

In [ ]:
id = hclin2pid(errors[30])
registros.loc[registros.NumeroHistoria == int(id)]

In [ ]:
casos.loc['AD326269']

In [ ]:
id = hclin2pid('1050005797')
casos.loc[casos.Paciente == id]

In [ ]:
j.VAR_0183

In [ ]:
for _, i in automatic.iterrows():
    if '1050005797' in i.ID01:
        print(i.ID01)

In [ ]:
automatic.VAR_0183